# Content Creation at Scale

## Initial Imports

In [41]:
# Warning control
import warnings
warnings.filterwarnings('ignore')



import os
from dotenv import load_dotenv
import yaml
from crewai import Agent, Task, Crew, LLM

In [42]:
load_dotenv()


## Creating Structured Output

In [43]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

we want to make sure the data is consistent when we passed them around to different agents and systems

In [44]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [45]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [46]:
websearchtool = WebsiteSearchTool(
    config=dict(
        llm=dict(
            provider="azure_openai", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gpt-4o-mini",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="azure_openai", # or openai, ollama, ...
            config=dict(
                model="text-embedding-ada-002",
                deployment_name= "text-embedding-ada-002",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)


## Creating Crew, Agents, and Tasks

In [36]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=LLM(model=f'azure/gpt-4o'),
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), websearchtool],
    llm=LLM(model=f'azure/gpt-4o'),
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), websearchtool], # had to customize WebsiteSearchTool
    llm=LLM(model=f'azure/gpt-4o')
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
    llm=LLM(model=f'azure/gpt-4o')
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

2025-01-13 15:35:16,979 - 140577126400384 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Kicking off the Crew

In [37]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.





# Agent: Lead Market Analyst
## Thought: To provide an accurate and detailed summary report highlighting the most impactful financial news and updates related to US inflation and its effect on the 2024 stock market, I must begin by searching for the latest news and data. I will focus on identifying recent trends, significant economic indicators, and any notable regulatory changes.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation impact on stock market 2024 news\"}"
## Tool Output: 
{'searchParameters': {'q': 'US inflation impact on stock market 2024 news', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Next CPI Inflation Report Could Reduce 2025 Fed Rate Cut ... - Forbes', 'link': 'https://www.forbes.com/sites/jasonschenker/2025/01/12/next-cpi-inflation-report-could-reduce-2025-fed-rate-cut-expectations/', 'snippet': 'U.S. year-on-year total CPI consumer inflation [+][-] accelerated in November 2024 to 2.7%. U.S. Bureau o

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]




# Agent: Chief Data Strategist
## Thought: Thought: I now have several sources that provide insights into US inflation trends and the stock market for 2024. I should select the most relevant sources to consolidate the data needed for a comprehensive analysis report.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"2024 US inflation trends and stock market impact\", \"website\": \"https://tradingeconomics.com/united-states/inflation-cpi\"}"
## Tool Output: 
Relevant Content:
United States Inflation Rate

Market Funds as the Fed Cuts Rates Wealth Management Rising Inflation May Challenge Stocks Wealth Management 3 Ways to Manage Your Cash More Strategically 7275232 63766 3CD19151-F46C-4595-84CF-8EEC3E05FD19 false false Sign up to get Morgan Stanley’s Five Ideas newsletter delivered weekly to your inbox. Email Address Email Address (Required) Invalid Email Address Submit Thank You for Subscribing! Thank you for your interest in Morgan Stanley Ideas. defa

## Social Content

In [38]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
🏛️ Inflation Alert: US inflation is projected to
stay above 2% in 2024! 📈 Here's how it impacts
your investments and what strategies you can
implement. #Inflation #InvestSmart
--------------------------------------------------
LinkedIn
Inflation's subtle yet powerful influence on the
market cannot be overlooked. With US inflation
rising steadily, explore strategies and sectors
that promise resilience and growth amidst
volatility. Dive into our full analysis here:
[Link to blog post] #InflationTrends
#StockMarketInsights
--------------------------------------------------
Instagram
Swipe up to explore the rising inflation trends in
the US 🌟 Understand its impact on your portfolio
and discover strategic opportunities for 2024!
[Link to blog post]
--------------------------------------------------
Facebook
📊 Stay ahead in the investment game with insights
into US inflation and stock market trends for
2024! Learn how technology stocks and inflation-
hedging strategies can bolster yo

## Blog Post

In [39]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Understanding Inflation and Its Impact in 2024: A Guide for Investors

## Introduction
As we progress through 2024, inflation remains a key focal point for economists and investors alike. With rising inflation rates affecting economic strategies and monetary policies, understanding its impact on the stock market is crucial for investors looking to navigate these uncertain waters. This blog post explores how inflation influences market dynamics, investor sentiment, and potential opportunities and risks.

## Inflation Trends and Economic Growth
In recent months, inflation in the US has seen a steady increase, with rates climbing to 2.7% in November 2024. The ongoing supply chain disruptions and strong consumer demand suggest that inflation will likely remain above the Federal Reserve's target of 2% throughout the year. Meanwhile, the economic growth forecast stands at 2.7% year-over-year, supported by a robust job market and a steady unemployment rate of 4.1%.

The persistent elevation of inflation has notable implications for economic growth, with businesses and consumers adapting to changing price levels. While businesses face cost pressures, they attempt to balance these by optimizing supply chains and improving efficiency. This dynamic significantly impacts consumer purchasing power, potentially leading to shifts in spending habits.

## Monetary Policy Implications
The persistent inflationary pressure has led the Federal Reserve to maintain a cautious stance. Despite market speculation, the Fed has paused interest rate cuts, opting instead to focus on stabilizing economic growth. There is anticipation surrounding potential rate adjustments in late 2025, dependent on how inflationary pressures evolve.

The Fed's current approach prioritizes long-term economic stability over short-term gains, aiming to mitigate abrupt market fluctuations. Investors must therefore monitor policy statements closely, as unexpected shifts could significantly alter market conditions and investment strategies.

## Stock Market Response
Despite concerns surrounding inflation, the stock market has demonstrated resilience, with major indices like the S&P 500 and Nasdaq posting gains. Technology stocks, in particular, have benefited from the current economic landscape. However, investors are urged to remain vigilant, especially in sectors sensitive to inflation, such as consumer goods and utilities, which may experience heightened volatility.

The bullish performance of the stock market, particularly in tech, is largely driven by continued innovation and demand for digital solutions. Nonetheless, the looming threat of prolonged inflation necessitates careful sector analysis and portfolio adjustments to safeguard against potential downturns in more vulnerable sectors.

## Investor Sentiment and Opportunities
Investor sentiment is cautiously optimistic, with equity rallies offering some reassurance amidst inflationary uncertainty. Opportunities abound in sectors like technology and innovation, known to perform well in inflationary periods. Additionally, strategies centered on hedging against inflation, such as investing in commodities and real estate, can be valuable considerations for investors.

While the optimistic outlook prevails, the cautious approach to investment ensures a balanced focus on risk mitigation and capitalizing on opportunities. Diversifying investments across diverse sectors and asset classes could further bolster resilience against market shifts driven by inflation.

## Conclusion: Adapting to Economic Dynamics
The intricate interplay between inflation trends and stock market performance in 2024 presents a complex yet navigable landscape for informed investors. By understanding and adapting to these economic dynamics, investors can optimize their portfolios and capitalize on emerging opportunities, positioning themselves for success in the year ahead.

Staying informed and agile will be key as economic conditions evolve, ensuring that investors are well-positioned to make strategic decisions in a rapidly changing environment.

## Visual Elements to Include:
- **Infographic:** A visual representation of inflation trends over the past months and anticipated projections for 2024.
- **Chart:** A graph showing the correlation between inflation rates and stock market performance.
- **Video:** A short explainer video summarizing the key insights into inflation's impact on the market.


In [40]:
import pandas as pd

costs = 2.50 * (content_creation_crew.usage_metrics.prompt_tokens + content_creation_crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([content_creation_crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0786


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,31444,27450,2432,3994,10
